In [1]:
import os
os.getcwd()
from tqdm import tqdm
import numpy as np
import cv2
import random

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from sklearn.preprocessing import LabelEncoder
import time
import tensorflow as tf

%matplotlib inline

/Users/midhunpk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
DIR = './../cat-dog-classifier/train/'


In [27]:
data = []
labels = []
IMG_SIZE = 64
for i,directory in enumerate(tqdm(os.listdir(DIR)[:2000])):
  try :
    path1 = os.path.join(DIR, directory)
    label = directory.split('.')[0]
    img = cv2.imread(path1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    labels.append([np.array(label)])
    data.append([np.array(img)])
  except :
    print(directory)
  if i == 2000:break
    
random.Random(1).shuffle(data)
random.Random(1).shuffle(labels)

100%|██████████| 2000/2000 [00:04<00:00, 442.24it/s]


In [28]:
train = np.array(data).reshape(-1,IMG_SIZE,IMG_SIZE,3)
lab= np.array(labels)

In [29]:
lab.shape

(2000, 1)

In [30]:
enc_lab = [1 if i == 'dog'else 0 for i in np.squeeze(lab) ]

In [31]:
labs = pd.get_dummies(enc_lab)


In [32]:
x_train = train[:1500,:,:,:]
x_test = train[1500:,:,:,:]

In [33]:
# labs

In [34]:
labs = np.array(labs)

In [35]:
y_train = labs[:1500, :]
y_test = labs[1500:,:]


In [36]:
y_train.shape

(1500, 2)

In [37]:
def create_placeholder(n_H0, n_W0, n_C0, n_y):
  X = tf.placeholder(tf.float32,[None,n_H0, n_W0, n_C0])
  Y = tf.placeholder(tf.float32,[None,n_y])
  return X,Y

In [38]:
def conv_layer(input, size_in, size_out, name='conv'):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev = 0.01), name = 'W')
    b = tf.Variable(tf.constant(0.01,shape=[size_out]),name = 'B')
    conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding ="SAME" )
    with tf.name_scope('relu'):
      act = tf.nn.relu(conv + b)
      tf.summary.histogram('weights',w)
      tf.summary.histogram('biases',b)
      tf.summary.histogram('activations',act)
    with tf.name_scope('max_pool'):
        max_pool = tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    return max_pool

In [39]:
def compute_cost(Z,Y):
  with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))
#     tf.summary_scalar('cost',cost)
    return cost

In [40]:
def random_minibatch(X, Y, minibatch_size = 8):#old one and not efficiant
  minibatches = []
  m = X.shape[0]
  
  
  permutation = list(np.random.permutation(m))
  suffled_X = X[permutation, :, :, :]
  shuffled_Y = Y[permutation:]
  
  
  
  num_complete_minibatches = math.floor(m/minibatch_size)
  for i in range(num_complete_minibatches):
    minibatch_x = shuffled_X[(i*minibatch_size):((I+1)*minibatch_size), :, :, :]
    minibatch_y = shuffled_Y[(i*minibatch_size):(i+1)*minibatch_size, :]
    minibatch = (minibatch_x,minibatch_y)
    minibatches.append(minibatch)
    
    if m%minibatch_size != 0:
      minibatch_x = shuffled_X[(num_complete_minibatches*minibatch_size):m, :, :, :]
      minibatch_y = shuffled_Y[(num_complete_minibatches*minibatch_size):m, :]
      minibatch = (minibatch_x,minibatch_y)
      minibatches.append(minibatch)
    yield minibatches

In [41]:
def iterate_minibatches(inputs,targets,batchsize):#using python operator
  assert inputs.shape[0] == targets.shape[0]
  m = inputs.shape[0]
  indices = np.arange(m)
  np.random.shuffle(indices)
  for index in range(0, m - batchsize + 1, batchsize):
    batch = indices[index:index+batchsize]
    yield inputs[batch], targets[batch]
    
  if m % batchsize != 0:
        batch = indices[math.floor(m/batchsize)*batchsize:m]
        yield inputs[batch], targets[batch]

In [42]:

(m, n_H0, n_W0, n_C0) = x_train.shape
n_y = y_train.shape[1]


X, Y = create_placeholder(n_H0, n_W0, n_C0, n_y)


In [43]:
X.shape

TensorShape([Dimension(None), Dimension(64), Dimension(64), Dimension(3)])

In [44]:
def model(X_train, X_test, Y_train, Y_test, learning_rate = 0.01, num_epoches = 2, batchsize = 50):
  tf.reset_default_graph() # clear the graph
  
  #iteration = tf.Variable(0, name="iteration", trainable=False)
  global_step = tf.Variable(0, name = 'global_step', trainable = False)
  
  
  (m, n_H0, n_W0, n_C0) = X_train.shape
  n_y = Y_train.shape[1]
  
  
  X, Y = create_placeholder(n_H0, n_W0, n_C0, n_y)
  
  
  #conv layers
  conv1 = conv_layer(X,     3, 10, 'conv1')
  conv2 = conv_layer(conv1, 10, 16, 'conv2')
#   conv3 = conv_layer(conv2, 16, 32, 'conv3')
#   conv4 = conv_layer(conv3, 32, 64, 'conv4')
#   conv5 = conv_layer(conv4, 64, 128, 'conv5')
  flattend = tf.contrib.layers.flatten(conv2)
  
  
  
  
  
  #FC layers
  
  fc1 = tf.contrib.layers.fully_connected(flattend, 128, activation_fn = None)
  Z = tf.contrib.layers.fully_connected(fc1, 2, activation_fn = None)
  prob = tf.nn.softmax(Z, name = 'probs')
  prediction = tf.argmax(Z, 1, name = 'predictions')
  
  
  cost = compute_cost(Z, Y)
  
  
  
  with tf.name_scope('optimize'):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost, global_step = global_step)
    
    
  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(Z,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     tf.summary.scalar('accuracy', accuracy)
    
    
    
  summaryMerged = tf.summary.merge_all()
  filename="./summary_log/run-"+time.strftime("%d%m-%H%M%S") 
  
  
  
  
  
#   saver = tf.train.saver()
  
  
  
  tf.add_to_collection('X', X)
  tf.add_to_collection('Z', Z)
  tf.add_to_collection('correct_prediction', correct_prediction)
  
  
  

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess,tf.train.latest_checkpoint('.\\saved_model'))
    writer = tf.summary.FileWriter(filename, sess.graph)
    
    
    
    batchsize = 10
    for epoch in tqdm(range(num_epoches)):
      avg_cost = 0
      
      for i, minibatch in tqdm(enumerate(iterate_minibatches(X_train, Y_train, batchsize))):
        
        (minibatch_X,minibatch_Y ) = minibatch
        
        sess.run(optimizer, feed_dict = {X: minibatch_X, Y: minibatch_Y})
        
        avg_cost += sess.run(cost, feed_dict = {X:minibatch_X, Y: minibatch_Y})
        
        iteration = sess.run(global_step)
        
        
        summ = sess.run(summaryMerged, feed_dict={X: minibatch_X, Y: minibatch_Y})
#         writer.add_summary(summ, (iteration))
        
        
      print('cost after iteration %i: %f' %(i, avg_cost/(i+1)))
      
    print('Training completed ! !')

    # save weights
#     saver.save(sess, "./saved_model/model")

#     Export meta graph
    tf.train.export_meta_graph(filename = './meta/model.meta')


    #model accuracy

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))*100
    train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
    test_accuracy = accuracy.eval({X:X_test, Y: Y_test})
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("Total iterations completed : ", iteration)



    #One step forwardprop
    predict_op = tf.argmax(Z, 1) 
    pred = sess.run(predict_op, feed_dict={X: x_test})
    
  return pred

In [46]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1500, 64, 64, 3), (1500, 2), (500, 64, 64, 3), (500, 2))

In [51]:
#no dropout
pred= model(x_train, x_test, y_train, y_test, num_epoches = 10, batchsize = 10, learning_rate = 0.0001)

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  5.66it/s]
3it [00:00,  9.90it/s]
5it [00:00, 11.59it/s]
7it [00:00, 12.65it/s]
9it [00:00, 13.22it/s]
11it [00:00, 13.63it/s]
13it [00:00, 14.00it/s]
15it [00:01, 14.29it/s]
17it [00:01, 14.56it/s]
19it [00:01, 14.73it/s]
21it [00:01, 14.92it/s]
23it [00:01, 15.08it/s]
25it [00:01, 15.21it/s]
27it [00:01, 15.31it/s]
29it [00:01, 15.40it/s]
31it [00:02, 15.46it/s]
33it [00:02, 15.53it/s]
35it [00:02, 15.60it/s]
37it [00:02, 15.65it/s]
39it [00:02, 15.70it/s]
41it [00:02, 15.72it/s]
43it [00:02, 15.76it/s]
45it [00:02, 15.80it/s]
47it [00:02, 15.80it/s]
49it [00:03, 15.83it/s]
51it [00:03, 15.85it/s]
53it [00:03, 15.88it/s]
55it [00:03, 15.91it/s]
57it [00:03, 15.94it/s]
59it [00:03, 15.97it/s]
61it [00:03, 15.98it/s]
63it [00:03, 15.99it/s]
65it [00:04, 16.01it/s]
67it [00:04, 16.03it/s]
69it [00:04, 16.05it/s]
71it [00:04, 16.05it/s]
73it [00:04, 16.07it/s]
75it [00:04, 16.08it/s]
77it [00:04, 16.08it/s]
79it [00:04

cost after iteration 149: 0.696334



4it [00:00, 16.06it/s]
6it [00:00, 16.18it/s]
8it [00:00, 16.19it/s]
10it [00:00, 16.23it/s]
12it [00:00, 16.26it/s]
14it [00:00, 16.27it/s]
16it [00:00, 16.29it/s]
18it [00:01, 16.31it/s]
20it [00:01, 16.22it/s]
22it [00:01, 16.14it/s]
24it [00:01, 16.10it/s]
26it [00:01, 16.10it/s]
28it [00:01, 16.13it/s]
30it [00:01, 16.12it/s]
32it [00:01, 16.12it/s]
34it [00:02, 16.16it/s]
36it [00:02, 16.19it/s]
38it [00:02, 16.22it/s]
40it [00:02, 16.21it/s]
42it [00:02, 16.19it/s]
44it [00:02, 16.19it/s]
46it [00:02, 16.21it/s]
48it [00:02, 16.20it/s]
50it [00:03, 16.21it/s]
52it [00:03, 16.13it/s]
54it [00:03, 16.09it/s]
56it [00:03, 15.97it/s]
58it [00:03, 15.95it/s]
60it [00:03, 15.88it/s]
62it [00:03, 15.79it/s]
64it [00:04, 15.80it/s]
66it [00:04, 15.83it/s]
68it [00:04, 15.85it/s]
70it [00:04, 15.87it/s]
72it [00:04, 15.89it/s]
74it [00:04, 15.90it/s]
76it [00:04, 15.92it/s]
78it [00:04, 15.93it/s]
80it [00:05, 15.94it/s]
82it [00:05, 15.97it/s]
84it [00:05, 15.98it/s]
86it [00:05, 16.01

cost after iteration 149: 0.637618



4it [00:00, 14.58it/s]
6it [00:00, 15.10it/s]
8it [00:00, 15.33it/s]
10it [00:00, 15.39it/s]
12it [00:00, 15.36it/s]
14it [00:00, 15.35it/s]
16it [00:01, 15.40it/s]
18it [00:01, 15.44it/s]
20it [00:01, 15.48it/s]
22it [00:01, 15.53it/s]
24it [00:01, 15.59it/s]
26it [00:01, 15.64it/s]
28it [00:01, 15.68it/s]
30it [00:01, 15.69it/s]
32it [00:02, 15.71it/s]
34it [00:02, 15.73it/s]
36it [00:02, 15.77it/s]
38it [00:02, 15.81it/s]
40it [00:02, 15.83it/s]
42it [00:02, 15.86it/s]
44it [00:02, 15.89it/s]
46it [00:02, 15.81it/s]
48it [00:03, 15.81it/s]
50it [00:03, 15.81it/s]
52it [00:03, 15.83it/s]
54it [00:03, 15.85it/s]
56it [00:03, 15.87it/s]
58it [00:03, 15.89it/s]
60it [00:03, 15.89it/s]
62it [00:03, 15.90it/s]
64it [00:04, 15.89it/s]
66it [00:04, 15.90it/s]
68it [00:04, 15.90it/s]
70it [00:04, 15.91it/s]
72it [00:04, 15.92it/s]
74it [00:04, 15.92it/s]
76it [00:04, 15.93it/s]
78it [00:04, 15.94it/s]
80it [00:05, 15.92it/s]
82it [00:05, 15.86it/s]
84it [00:05, 15.80it/s]
86it [00:05, 15.74

...

cost after iteration 149: 0.262438



4it [00:00, 16.22it/s]
6it [00:00, 16.21it/s]
8it [00:00, 16.21it/s]
10it [00:00, 16.21it/s]
12it [00:00, 16.22it/s]
14it [00:00, 16.22it/s]
16it [00:00, 16.17it/s]
18it [00:01, 16.19it/s]
20it [00:01, 16.18it/s]
22it [00:01, 16.18it/s]
24it [00:01, 16.20it/s]
26it [00:01, 16.22it/s]
28it [00:01, 16.22it/s]
30it [00:01, 16.24it/s]
32it [00:01, 16.26it/s]
34it [00:02, 16.28it/s]
36it [00:02, 16.28it/s]
38it [00:02, 16.27it/s]
40it [00:02, 16.27it/s]
42it [00:02, 16.27it/s]
44it [00:02, 16.27it/s]
46it [00:02, 16.27it/s]
48it [00:02, 16.28it/s]
50it [00:03, 16.28it/s]
52it [00:03, 16.29it/s]
54it [00:03, 16.29it/s]
56it [00:03, 16.29it/s]
58it [00:03, 16.29it/s]
60it [00:03, 16.29it/s]
62it [00:03, 16.30it/s]
64it [00:03, 16.30it/s]
66it [00:04, 16.28it/s]
68it [00:04, 16.27it/s]
70it [00:04, 16.26it/s]
72it [00:04, 16.26it/s]
74it [00:04, 16.27it/s]
76it [00:04, 16.27it/s]
78it [00:04, 16.27it/s]
80it [00:04, 16.28it/s]
82it [00:05, 16.28it/s]
84it [00:05, 16.29it/s]
86it [00:05, 16.30

cost after iteration 149: 0.216965



4it [00:00, 15.97it/s]
6it [00:00, 16.11it/s]
8it [00:00, 16.08it/s]
10it [00:00, 16.13it/s]
12it [00:00, 16.12it/s]
14it [00:00, 16.10it/s]
16it [00:00, 16.10it/s]
18it [00:01, 16.13it/s]
20it [00:01, 16.10it/s]
22it [00:01, 16.09it/s]
24it [00:01, 16.03it/s]
26it [00:01, 16.05it/s]
28it [00:01, 16.07it/s]
30it [00:01, 16.07it/s]
32it [00:01, 16.09it/s]
34it [00:02, 16.10it/s]
36it [00:02, 16.11it/s]
38it [00:02, 16.11it/s]
40it [00:02, 16.12it/s]
42it [00:02, 16.14it/s]
44it [00:02, 16.14it/s]
46it [00:02, 16.15it/s]
48it [00:02, 16.16it/s]
50it [00:03, 16.17it/s]
52it [00:03, 16.18it/s]
54it [00:03, 16.19it/s]
56it [00:03, 16.20it/s]
58it [00:03, 16.19it/s]
60it [00:03, 16.20it/s]
62it [00:03, 16.20it/s]
64it [00:03, 16.20it/s]
66it [00:04, 16.21it/s]
68it [00:04, 16.21it/s]
70it [00:04, 16.22it/s]
72it [00:04, 16.22it/s]
74it [00:04, 16.22it/s]
76it [00:04, 16.23it/s]
78it [00:04, 16.23it/s]
80it [00:04, 16.22it/s]
82it [00:05, 16.22it/s]
84it [00:05, 16.22it/s]
86it [00:05, 16.22

cost after iteration 149: 0.181288
Training completed ! !
Train Accuracy: 94.73334
Test Accuracy: 64.600006
Total iterations completed :  1500
